<a href="https://colab.research.google.com/github/HK-Victor-AI/APAC-2023-Workshop/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****************************************************
####################################################

**Stage 1:
Download a PDF file and create vector store**
####################################################
****************************************************

In [2]:
# GET GPU FIRST !!!!!
# T4 GPU

# first we download a/some pdf files
# For simplicity, we just download and process one pdf file. You can create a folder and place your pdf files there.

!wget https://github.com/HK-Victor-AI/APAC-2023-Workshop/raw/main/criminal_procedure_code.pdf -O criminal_procedure_code.pdf

--2023-10-08 07:16:56--  https://github.com/HK-Victor-AI/APAC-2023-Workshop/raw/main/criminal_procedure_code.pdf
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/HK-Victor-AI/APAC-2023-Workshop/main/criminal_procedure_code.pdf [following]
--2023-10-08 07:16:57--  https://raw.githubusercontent.com/HK-Victor-AI/APAC-2023-Workshop/main/criminal_procedure_code.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1574942 (1.5M) [application/octet-stream]
Saving to: ‘criminal_procedure_code.pdf’

criminal_procedure_ 100%[===================>]   1.50M  --.-KB/s    in 0.04s   

2023-10-08 07:16:57 (35.5 M

In [3]:
!ls -l

total 1544
-rw-r--r-- 1 root root 1574942 Oct  8 07:16 criminal_procedure_code.pdf
drwxr-xr-x 1 root root    4096 Oct  5 19:33 sample_data


In [4]:
!pip install langchain==0.0.310
!export LLAMA_CUBLAS=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python==0.2.11
!pip install sentence-transformers==2.2.2
!pip install chromadb==0.3.26
!pip install unstructured==0.10.19 pdf2image==1.16.3 pdfminer==20191125
!pip install glob2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423539 sha256=393b722794ae2b96f4be9dfb1098c86fc659202c47058bae0fabe40844043afe
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Prepar

In [1]:
!pip uninstall pdfminer
!pip install pdfminer.six
!pip install unstructured==0.10.19
#from pdfminer.pdfinterp import PDFResourceManager
#from pdfminer.converter import PDFPageAggregator

In [2]:
import chromadb
from typing import List
from chromadb.config import Settings
from langchain.document_loaders import (
    TextLoader,
    UnstructuredPDFLoader,

)

LOADER_MAPPING = {
    ".txt": (TextLoader, {}),       # text loader class, no parameter
    ".pdf": (UnstructuredPDFLoader, {}),      # PDF loader class, no parameter

}

In [3]:
# Load one document and save to a document object
from langchain.docstore.document import Document
import glob

def load_one_document(file_path):
  # get file extension
  file_ext = "." + file_path.rsplit(".", 1)[-1]

  # check whether the file type is in LOADER_MAPPING
  #   if YES, Load the file content to "Document" object
  #   if NO, skip
  docs_data = None
  if file_ext in LOADER_MAPPING:
    loader_class, loader_args = LOADER_MAPPING[file_ext]
    loader = loader_class(file_path, **loader_args)
    docs_data = loader.load()
  return docs_data


# Load all documents
import os
def load_all_documents(source_dir):
  all_files = glob.glob(source_dir + '/' + '*.*')
  all_docs_data = []
  for one_file in all_files:
    all_docs_data.extend(load_one_document(one_file))
  return all_docs_data


All files are now loaded into document objects. The next step is to split the contents hold in document objects into chunks (smaller pieces of text).

In [5]:
# source_directry is the path where input files are there
from langchain.text_splitter import RecursiveCharacterTextSplitter

source_directory = "/content"
chunk_size = 1000
chunk_overlap = 100

# load files and split contents
all_docs_data = load_all_documents(source_directory)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunked_docs = text_splitter.split_documents(all_docs_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
chunked_docs[110]

Document(page_content='Form of search warrant 26.—(1) A search warrant issued by a court under this Code must be in writing bearing the seal of the court, and signed by a Magistrate or District Judge (as the case may be), or in the case of the General Division of the High Court, by a Judge or by the Registrar of the Supreme Court.\n\n[40/2019]\n\nInformal Consolidation – version in force from 26/6/2023\n\n2020 Ed.\n\nCriminal Procedure Code 2010\n\n52\n\n(2) A search warrant must ordinarily be issued to the Commissioner of Police and to one or more other police officers to be designated by name in the warrant, and all or any of those police officers may execute it.\n\n(3) The court may in appropriate circumstances issue a search warrant to one or more named persons who are not police officers, and all or any of those persons may execute it.', metadata={'source': '/content/criminal_procedure_code.pdf'})

In [8]:
# Create embedding function

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from chromadb.config import Settings


# configuration of the vector store
# persist_directory = "./"

CHROMA_SETTINGS = Settings(
    chroma_db_impl = 'duckdb+parquet',  # use duckdb to r/w parquet file
    anonymized_telemetry = False     # refuse telemetry of usage info
)
model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}      # you must have a gpu, otherwise change it to cpu
encode_kwargs = {'normalize_embeddings': True}
embedding_func = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

print("Creating new vector store...")
db = Chroma.from_documents(
    chunked_docs,
    embedding_func,
    persist_directory=persist_directory,
    client_settings=CHROMA_SETTINGS
)

db.persist()
db = None
print("Files ingestion completed!")

Creating new vector store...
Files ingestion completed!


In [9]:
!ls -l

total 4716
-rw-r--r-- 1 root root     557 Oct  8 07:33 chroma-collections.parquet
-rw-r--r-- 1 root root 3238591 Oct  8 07:33 chroma-embeddings.parquet
-rw-r--r-- 1 root root 1574942 Oct  8 07:16 criminal_procedure_code.pdf
drwxr-xr-x 2 root root    4096 Oct  8 07:33 index
drwxr-xr-x 1 root root    4096 Oct  5 19:33 sample_data


In [10]:
ls -l index

total 2076
-rw-r--r-- 1 root root   38945 Oct  8 07:33 id_to_uuid_e403fbb6-b2b2-4add-8c16-c2dd00cef57a.pkl
-rw-r--r-- 1 root root 2029800 Oct  8 07:33 index_e403fbb6-b2b2-4add-8c16-c2dd00cef57a.bin
-rw-r--r-- 1 root root     105 Oct  8 07:33 index_metadata_e403fbb6-b2b2-4add-8c16-c2dd00cef57a.pkl
-rw-r--r-- 1 root root   45553 Oct  8 07:33 uuid_to_id_e403fbb6-b2b2-4add-8c16-c2dd00cef57a.pkl


****************************************************
####################################################

**Stage 2:
Input files ingested and chromadb is created!
Now we go to the inference stage.**
####################################################
****************************************************

In [11]:
# This is the chat model!
!wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf

--2023-10-08 07:33:48--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1697007960&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzAwNzk2MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84ZGIxZDFmNzNiNGNhYTU4ZTk0N2NjYmZlMmZiMjdhYzVlNDk1YzJhZDg0NTdhZDI5OWQxNTk4N2FlZTNiNTIwL2VkYTJhMTVkNTMyYmVhNGNlNmZjMTRkMTVjMmI3MjYzODI0M

In [12]:
from langchain.llms import LlamaCpp
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    n_ctx = 4096,
    model_path="/content/llama-2-13b-chat.Q4_0.gguf", # location of the model
    temperature=0.75,                 # temperature
    max_tokens=2000,                 # Max. number of tokens to be generated
    top_p=0.9,                    # top_p = 0.9
    top_k=30,                     # top_k = 30
    n_gpu_layers=43,                 # number of layers to offload to GPU
    verbose=True, # Verbose is required to pass to the callback manager
    n_batch=200,          # number of token generation in parallel
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [13]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant."

def create_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
  SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
  prompt = B_INST + SYSTEM_PROMPT + instruction + E_INST
  return "<s>" + prompt

In [14]:
# open the vector database
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.config import Settings
from langchain.vectorstores import Chroma


model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}      # you must have a gpu, otherwise change it to cpu
encode_kwargs = {'normalize_embeddings': True}
embedding_func = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)


persist_directory = "./"

CHROMA_SETTINGS = Settings(
    chroma_db_impl = 'duckdb+parquet',  # use duckdb to r/w parquet file
    anonymized_telemetry = False     # refuse telemetry of usage info
)

vector_db = Chroma(persist_directory=persist_directory,
            embedding_function=embedding_func,
            client_settings=CHROMA_SETTINGS)

In [15]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


instruction = """Given the following passage, you find out the answer to the question.

Passage: {context}

Question: {question}

Helpful answer: """

prompt_template = create_prompt(instruction)
PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

chain_type_kwargs = {"prompt": PROMPT}
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

query = "what are the sentences other than death?"
relevant_docs = retriever.get_relevant_documents(query)
print(relevant_docs)



[Document(page_content='314. A sentence of death must not be passed or recorded against an accused convicted of an offence if the court has reason to believe that, at the time the offence was committed, the accused was below 18 years of age, but instead the court must sentence the accused to life imprisonment. Sentence of death not to be passed on pregnant woman 315.—(1) Where a woman convicted of an offence punishable with death alleges that she is pregnant, or where the court before whom a woman is so convicted thinks fit, the question whether or not the woman is pregnant must, before sentence is passed on her, be determined by the court.\n\n(2) If the court finds the woman pregnant, it must pass a sentence of\n\nlife imprisonment on her.\n\n(3) If the court finds the woman not to be pregnant, she may appeal to the Court of Appeal against that finding in the manner set out under this Code.', metadata={'source': '/content/criminal_procedure_code.pdf'}), Document(page_content='[19/2018

In [25]:
# Q&A Method 1 : Use langchain RetrievalQA chain

qa_chain = RetrievalQA.from_chain_type(
                          llm=llm,
                          chain_type="stuff",
                          retriever=retriever,
                          return_source_documents=True,
                          chain_type_kwargs=chain_type_kwargs,
                      )
answer = qa_chain(query)
print("Question: ", answer['query'] + '\n\n')
print("Answer:", answer['result'] + '\n\n')
answer['source_documents']

Llama.generate: prefix-match hit


Question:  what are the sentences other than death?


Answer:   Based on the passage provided, the sentences other than death are:

1. Life imprisonment
2. Imprisonment for a term not exceeding 14 years (as specified in the Sixth Schedule)

The passage states that if an accused is convicted of an offence punishable with death and the court has reason to believe that the accused was below 18 years of age at the time the offence was committed, the court must sentence the accused to life imprisonment instead of death. Additionally, if a woman convicted of an offence punishable with death alleges that she is pregnant, the court must pass a sentence of life imprisonment on her, regardless of whether or not she is actually pregnant.




[Document(page_content='314. A sentence of death must not be passed or recorded against an accused convicted of an offence if the court has reason to believe that, at the time the offence was committed, the accused was below 18 years of age, but instead the court must sentence the accused to life imprisonment. Sentence of death not to be passed on pregnant woman 315.—(1) Where a woman convicted of an offence punishable with death alleges that she is pregnant, or where the court before whom a woman is so convicted thinks fit, the question whether or not the woman is pregnant must, before sentence is passed on her, be determined by the court.\n\n(2) If the court finds the woman pregnant, it must pass a sentence of\n\nlife imprisonment on her.\n\n(3) If the court finds the woman not to be pregnant, she may appeal to the Court of Appeal against that finding in the manner set out under this Code.', metadata={'source': '/content/criminal_procedure_code.pdf'}),
 Document(page_content='[19/201